In [2]:
import numpy as np
import time
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import XTC, GRO
import MDAnalysis.analysis.rms
from MDAnalysis.analysis.rms import rmsd
from MDAnalysis.analysis import align
import glob
################################################################################################################
#####-------residue numbers for MEK kinase-----#####f
alpha_start = 107
alpha_end = 125
actloop_start = 208
actloop_end = 233
#############################


var_xtc = []
for i in range(0,10): # number of HILLS files
    myPath = '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/trajectory_files/traj_curr/traj_wt/'
    tifCounter = len(glob.glob1(myPath + 'trajectories_'+str(i),"*.xtc"))
    for j in range(0,tifCounter):
        var_xtc.append( myPath + 'trajectories_'+str(i) +'/' +glob.glob1(myPath + 'trajectories_'+str(i),"*.xtc")[j])

    
################################################################################################################ 
################################################################################################################
var_gro = '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/trajectory_files/active_mek_wt_protein.gro' 
u = mda.Universe(var_gro, var_xtc)



In [3]:

##############################################################################################################
with open('/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/free_ener_wt/8us/fes_8us_wt_mintozero.dat') as f:
    
    lines = f.readlines()
    
    
    
#lines = lines[9:2659]

new_lines = []    
# Get rid of empty lines
for line in lines:
    # Strip whitespace, should leave nothing if empty line was just "\n"
    if not line.strip():
        continue
    # We got something, save it
    else:
        new_lines.append(line)
        
lines = new_lines[9:]


bin_size_active = 201 # This was set in the PLUMED simulation
bin_size_inactive = 201
x = []
y1 = [] # stores the active coordinates
y2 = [] # stores the inactive coordinates
z1 = []
z = np.zeros((1,bin_size_inactive))
for i in range(0,len(lines)):
    
    words = lines[i].split()
    int_lst = [float(x) for x in words]
   
    y1.append(int_lst[0]) #y1 takes in active rmsd coordinates
    y2.append(int_lst[1]) #y2 takes in inactive rmsd coordinates
    z1.append(int_lst[2])  #z takes the free energy
 
            
y1 = np.array(y1)
y2 = np.array(y2)

y1 = y1[0:bin_size_inactive]
y2 = y2[0:len(lines):bin_size_inactive]

bin_size_active = 201
bin_size_inactive = 201

bin_width1 = (y1.max() - y1.min()) / (bin_size_active)   # y1max - min
bin_width2 =  (y2.max() - y2.min()) / (bin_size_inactive) # y2max - min

In [4]:
len(u.trajectory)


260040

In [7]:
start = time.time()
active_ref1 = '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/trajectory_files/mek_active_CA.pdb'
inactive_ref1 = '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/trajectory_files/mek_inactive_CA.pdb'
active_ref = mda.Universe(active_ref1)
inactive_ref = mda.Universe(inactive_ref1)

protein = u.select_atoms("protein")

with MDAnalysis.Writer("zone1.xtc", protein.n_atoms) as Z1, MDAnalysis.Writer("zone2.xtc", protein.n_atoms) as Z2, MDAnalysis.Writer("zone3.xtc", protein.n_atoms) as Z3, MDAnalysis.Writer("zone4.xtc", protein.n_atoms) as Z4, MDAnalysis.Writer("zone5.xtc", protein.n_atoms) as Z5, MDAnalysis.Writer("zone6.xtc", protein.n_atoms) as Z6, MDAnalysis.Writer("zone7.xtc", protein.n_atoms) as Z7, MDAnalysis.Writer("zone8.xtc", protein.n_atoms) as Z8:                                             
    
        
    for j in range(0,len(u.trajectory)):
        u.trajectory[j]



        align.alignto(u, inactive_ref, select="protein and not(resid {}:{}) and not(resid {}:{})  and name CA".format(alpha_start,alpha_end,actloop_start,actloop_end), weights="mass")

        a1helixloopi  = inactive_ref.select_atoms("(resid {}:{} or resid {}:{}) and name CA".format(alpha_start,alpha_end,actloop_start,actloop_end))
        a1CAhelixloopi  = a1helixloopi.positions

        a2helixloopi   = u.select_atoms("(resid {}:{} or resid {}:{}) and name CA".format(alpha_start,alpha_end,actloop_start,actloop_end))
        a2CAhelixloopi  = a2helixloopi.positions


        rmsd_inactive = rmsd(a1CAhelixloopi, a2CAhelixloopi)


        align.alignto(u, active_ref, select="protein and not(resid {}:{}) and not(resid {}:{})  and name CA".format(alpha_start,alpha_end,actloop_start,actloop_end), weights="mass")

        a1helixloopa  = active_ref.select_atoms("(resid {}:{} or resid {}:{}) and name CA".format(alpha_start,alpha_end,actloop_start,actloop_end))
        a1CAhelixloopa  = a1helixloopa.positions

        a2helixloopa   = u.select_atoms("(resid {}:{} or resid {}:{}) and name CA".format(alpha_start,alpha_end,actloop_start,actloop_end))
        a2CAhelixloopa  = a2helixloopa.positions

        rmsd_active = rmsd(a1CAhelixloopa, a2CAhelixloopa)

        
        #  Allocating the computed RMSD's to grid numbers
        g_x = np.math.ceil((rmsd_active - y1[0]) / bin_width1)
        g_y = np.math.ceil((rmsd_inactive - y2[0]) / bin_width2)
        
        
        c_x_z_1 = list(np.loadtxt('x_c_subset_bin_zone1.txt'))
        c_y_z_1 = list(np.loadtxt('y_c_subset_bin_zone1.txt'))
        c_stack_z_1 = np.dstack((c_x_z_1, c_y_z_1))
        
        c_x_z_2 = list(np.loadtxt('x_c_subset_bin_zone2.txt'))
        c_y_z_2 = list(np.loadtxt('y_c_subset_bin_zone2.txt'))
        c_stack_z_2 = np.dstack((c_x_z_2, c_y_z_2))
        
        c_x_z_3 = list(np.loadtxt('x_c_subset_bin_zone3.txt'))
        c_y_z_3 = list(np.loadtxt('y_c_subset_bin_zone3.txt'))
        c_stack_z_3 = np.dstack((c_x_z_3, c_y_z_3))
        
        c_x_z_4 = list(np.loadtxt('x_c_subset_bin_zone4.txt'))
        c_y_z_4 = list(np.loadtxt('y_c_subset_bin_zone4.txt'))
        c_stack_z_4 = np.dstack((c_x_z_4, c_y_z_4))
        
        c_x_z_5 = list(np.loadtxt('x_c_subset_bin_zone5.txt'))
        c_y_z_5 = list(np.loadtxt('y_c_subset_bin_zone5.txt'))
        c_stack_z_5 = np.dstack((c_x_z_5, c_y_z_5))
        
        
        c_x_z_6 = list(np.loadtxt('x_c_subset_bin_zone6.txt'))
        c_y_z_6 = list(np.loadtxt('y_c_subset_bin_zone6.txt'))
        c_stack_z_6 = np.dstack((c_x_z_6, c_y_z_6))
        
        
        c_x_z_7 = list(np.loadtxt('x_c_subset_bin_zone7.txt'))
        c_y_z_7 = list(np.loadtxt('y_c_subset_bin_zone7.txt'))
        c_stack_z_7 = np.dstack((c_x_z_7, c_y_z_7))
        
        c_x_z_8 = list(np.loadtxt('x_c_subset_bin_zone8.txt'))
        c_y_z_8 = list(np.loadtxt('y_c_subset_bin_zone8.txt'))
        c_stack_z_8 = np.dstack((c_x_z_8, c_y_z_8))

    ######################################################################################################################
    #####################################################################################################################


    #################-------ZONE1.xtc

        if g_x in c_x_z_1 and g_y in c_y_z_1:
            if [g_x,g_y] in c_stack_z_1[0].tolist():
                Z1.write(protein)
            
                     
    #################-------ZONE2.xtc
        if g_x in c_x_z_2 and g_y in c_y_z_2:
            if [g_x,g_y] in c_stack_z_2[0].tolist():
                Z2.write(protein)
            

    #################-------ZONE3.xtc
        if g_x in c_x_z_3 and g_y in c_y_z_3:
            if [g_x,g_y] in c_stack_z_3[0].tolist():
                Z3.write(protein)
    
    ################-------ZONE4.xtc
        if g_x in c_x_z_4 and g_y in c_y_z_4:
            if [g_x,g_y] in c_stack_z_4[0].tolist():
                Z4.write(protein)
    
     #################-------ZONE5.xtc
        if g_x in c_x_z_5 and g_y in c_y_z_5:
            if [g_x,g_y] in c_stack_z_5[0].tolist():
                Z5.write(protein)
                
      #################-------ZONE6.xtc
        if g_x in c_x_z_6 and g_y in c_y_z_6:
            if [g_x,g_y] in c_stack_z_6[0].tolist():
                Z6.write(protein)
                
       #################-------ZONE7.xtc
        if g_x in c_x_z_7 and g_y in c_y_z_7:
            if [g_x,g_y] in c_stack_z_7[0].tolist():
                Z7.write(protein)

        #################-------ZONE7.xtc
        if g_x in c_x_z_8 and g_y in c_y_z_8:
            if [g_x,g_y] in c_stack_z_8[0].tolist():
                Z8.write(protein)

/Users/owner/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:330: UserWarning: Element information is absent or missing for a few atoms. Elements attributes will not be populated.
  warnings.warn("Element information is absent or missing for a few "
/Users/owner/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/core/selection.py:521: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros(len(group), dtype=np.bool)


In [ ]:
%matplotlib inline
import numpy as np
import time
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import XTC, GRO
import MDAnalysis.analysis.rms
from MDAnalysis.analysis.rms import rmsd
from MDAnalysis.analysis import align
from MDAnalysis.analysis.dihedrals import Dihedral as dihedral
var_xtc = '/Volumes/KSPAT 3/Projects/Stas/metad_convergence/wt/free_ener_wt/8us/zone1.xtc'
var_gro = '/Volumes/KSPAT 3/Projects/Stas/metad_convergence/wt/trajectory_files/active_mek_wt_protein.gro' 
u = mda.Universe(var_gro, var_xtc)

selection = 'resname {} and resid {} and name {}'
ALA2N = selection.format('ASP', 208, 'CA')
ALA2CA = selection.format('ASP', 208, 'CG')
ALA2C = selection.format('CYS', 207, 'CA')
TYR3N = selection.format('CYS', 207, 'CB')
psi = [ALA2N, ALA2CA, ALA2C, TYR3N]

psi_angle = sum([u.select_atoms(atom) for atom in psi])  # sum of Atoms creates an AtomGroup
psi_angle = psi_angle.dihedral  # convert AtomGroup to Dihedral object

psi_list = []
for ts in u.trajectory:
    psi_list.append(psi_angle.value())
    
    
import matplotlib
import numpy as np
import matplotlib.pyplot as plt


psi_list = np.array(psi_list)

num_bins = 50

fig, ax = plt.subplots()

# the histogram of the data
n, bins, patches = ax.hist(psi_list, num_bins, density=True,color = "salmon")



#ax.plot(bins, y, '--')
ax.set_xlabel('Dihedral angle in deg',fontsize=18)
ax.set_ylabel('Frequency density',fontsize=18)
ax.set_title('DFG histogram',fontsize=18)
#plt.xlim((0,10))
plt.xticks(fontsize= 14)
#plt.yticks(fontsize= 20)
# Tweak spacing to prevent clipping of ylabel
fig.tight_layout()
plt.savefig('DFG_z1.png', bbox_inches = 'tight', dpi = 300)
plt.show()

In [ ]:
psi_list

In [ ]:
%matplotlib inline
import numpy as np
import time
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import XTC, GRO
import MDAnalysis.analysis.rms
from MDAnalysis.analysis.rms import rmsd
from MDAnalysis.analysis import align
from MDAnalysis.analysis.dihedrals import Dihedral as dihedral
var_xtc = '/Volumes/KSPAT 3/Projects/Stas/metad_convergence/wt/free_ener_wt/8us/zone8.xtc'
var_gro = '/Volumes/KSPAT 3/Projects/Stas/metad_convergence/wt/trajectory_files/active_mek_wt_protein.gro' 
u = mda.Universe(var_gro, var_xtc)

selection = 'resname {} and resid {} and name {}'
ALA2N = selection.format('ASP', 208, 'CA')
ALA2CA = selection.format('ASP', 208, 'CG')
ALA2C = selection.format('CYS', 207, 'CA')
TYR3N = selection.format('CYS', 207, 'CB')
psi = [ALA2N, ALA2CA, ALA2C, TYR3N]

psi_angle = sum([u.select_atoms(atom) for atom in psi])  # sum of Atoms creates an AtomGroup
psi_angle = psi_angle.dihedral  # convert AtomGroup to Dihedral object

psi_list = []
for ts in u.trajectory:
    psi_list.append(psi_angle.value())
    
import matplotlib
import numpy as np
import matplotlib.pyplot as plt


psi_list = np.array(psi_list)

num_bins = 50

fig, ax = plt.subplots()

# the histogram of the data
n, bins, patches = ax.hist(psi_list, num_bins, density=True,color = "salmon")



#ax.plot(bins, y, '--')
ax.set_xlabel('Dihedral angle in deg',fontsize=18)
ax.set_ylabel('Frequency density',fontsize=18)
ax.set_title('DFG histogram',fontsize=18)
#plt.xlim((-180,100))
plt.xticks(fontsize= 14)
#plt.yticks(fontsize= 20)
# Tweak spacing to prevent clipping of ylabel
fig.tight_layout()
plt.savefig('DFG_z8.png', bbox_inches = 'tight', dpi = 300)
plt.show()

In [ ]:
psi_list

In [ ]:
import numpy as np
import time
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import XTC, GRO
import MDAnalysis.analysis.rms
from MDAnalysis.analysis.rms import rmsd
from MDAnalysis.analysis import align
from MDAnalysis.analysis.dihedrals import Dihedral as dihedral
var_xtc = '/Volumes/KSPAT 3/Projects/Stas/metad_convergence/wt/free_ener_wt/8us/zone3.xtc'
var_gro = '/Volumes/KSPAT 3/Projects/Stas/metad_convergence/wt/trajectory_files/active_mek_wt_protein.gro' 
u = mda.Universe(var_gro, var_xtc)

selection = 'resname {} and resid {} and name {}'
ALA2N = selection.format('ASP', 208, 'CA')
ALA2CA = selection.format('ASP', 208, 'CG')
ALA2C = selection.format('CYS', 207, 'CA')
TYR3N = selection.format('CYS', 207, 'CB')
psi = [ALA2N, ALA2CA, ALA2C, TYR3N]

psi_angle = sum([u.select_atoms(atom) for atom in psi])  # sum of Atoms creates an AtomGroup
psi_angle = psi_angle.dihedral  # convert AtomGroup to Dihedral object

psi_list = []
for ts in u.trajectory:
    psi_list.append(psi_angle.value())
    
import matplotlib
import numpy as np
import matplotlib.pyplot as plt


psi_list = np.array(psi_list)

num_bins = 50

fig, ax = plt.subplots()

# the histogram of the data
n, bins, patches = ax.hist(psi_list, num_bins, density=True,color = "salmon")



#ax.plot(bins, y, '--')
ax.set_xlabel('Dihedral angle in deg',fontsize=18)
ax.set_ylabel('Frequency density',fontsize=18)
ax.set_title('DFG histogram',fontsize=18)
#plt.xlim((-180,100))
plt.xticks(fontsize= 14)
#plt.yticks(fontsize= 20)
# Tweak spacing to prevent clipping of ylabel
fig.tight_layout()
plt.savefig('DFG_z3.png', bbox_inches = 'tight', dpi = 300)
plt.show()

In [ ]:
 psi_list

In [11]:
import numpy as np
import time
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import XTC, GRO
import MDAnalysis.analysis.rms
from MDAnalysis.analysis.rms import rmsd
from MDAnalysis.analysis import align
import glob



var_xtc = []

myPath = '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/free_ener_wt/8us'
tifCounter = len(glob.glob1(myPath, "*.xtc"))
for j in range(0,tifCounter):
    var_xtc.append( myPath + '/' +glob.glob1(myPath,"*.xtc")[j])

    
################################################################################################################ 
################################################################################################################
var_gro = '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/trajectory_files/active_mek_wt_protein.gro' 
u = mda.Universe(var_gro, var_xtc)




In [12]:
var_xtc

['/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/free_ener_wt/8us/zone1.xtc',
 '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/free_ener_wt/8us/zone2.xtc',
 '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/free_ener_wt/8us/zone3.xtc',
 '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/free_ener_wt/8us/zone4.xtc',
 '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/free_ener_wt/8us/zone5.xtc',
 '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/free_ener_wt/8us/zone6.xtc',
 '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/free_ener_wt/8us/zone7.xtc',
 '/Volumes/3KSPAT/from_2KSPAT/metad_convergence/wt/free_ener_wt/8us/zone8.xtc']

In [13]:
len(u.trajectory)

33902

In [14]:
protein = u.select_atoms("protein")
with MDAnalysis.Writer("protein.xtc", protein.n_atoms) as W:
    for ts in u.trajectory:
        W.write(protein)